In [1]:
import numpy as np
import pandas as pd 
import sklearn
from sklearn.metrics  import f1_score,accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
# Classification
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost.sklearn import XGBClassifier
#
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.metrics import *
from sklearn.model_selection import *
# from catboost import CatBoost,CatBoostRegressor
from xgboost import XGBRegressor,XGBRFRegressor
np.random.seed(42)
sklearn.random.seed(42)

/usr/local/lib/python3.8/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.5.2.1-4-gc0b48a7 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.13.1-unknown is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.23ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.1.36ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(


In [2]:
data = pd.read_csv('./data/train.csv')

In [3]:
data = data.sample(frac=1)

In [4]:
test_data = pd.read_csv('./data/test.csv')

In [5]:
data['PassengerId'] = data['PassengerId'].astype(int)
data['Age'] = data['Age'].fillna(data['Age'].mean())
data['RoomService'] = data['RoomService'].fillna(data['RoomService'].mean())
data['FoodCourt'] = data['FoodCourt'].fillna(data['FoodCourt'].mean())
data['ShoppingMall'] = data['ShoppingMall'].fillna(data['ShoppingMall'].mean())
data['Spa'] = data['Spa'].fillna(data['Spa'].mean())
data['VRDeck'] = data['VRDeck'].fillna(data['VRDeck'].mean())

In [6]:
new_transported = []
for transport in data['Transported']:
    if transport is True:
        new_transported.append(0)
    elif transport is False:
        new_transported.append(1)
    else:
        new_transported.append(None)
data['Transported'] = new_transported

In [7]:
new_transported = []
for transport in data['CryoSleep']:
    if transport is True:
        new_transported.append(0)
    elif transport is False:
        new_transported.append(1)
    else:
        new_transported.append(None)
data['CryoSleep'] = new_transported

In [8]:
new_transported = []
for transport in data['VIP']:
    if transport is True:
        new_transported.append(0)
    elif transport is False:
        new_transported.append(1)
    else:
        new_transported.append(None)
data['VIP'] = new_transported

In [9]:
data = data.drop(['PassengerId','VIP','FoodCourt','ShoppingMall'],axis=1)

In [10]:
idx = 0
labels = {}
for label in data['HomePlanet']:
    if label not in list(labels.keys()):
        idx += 1
        labels[label] = idx

In [11]:
new_data = []
for i in data['HomePlanet']:
    new_data.append(labels[i])
data['HomePlanet'] = new_data

In [12]:
new_cabins = []
for c in data['Cabin']:
    if type(c) is float:
        new_cabins.append(None)
    else:
        c = c.split('/')[1]
        new_cabins.append(int(c))
data['Cabin'] = new_cabins
data['Cabin'] = data['Cabin'].fillna(data['Cabin'].mean())

In [13]:
data['CryoSleep'] = data['CryoSleep'].fillna(data['CryoSleep'].mean())

In [14]:
data = data.drop(['Destination','Name'],axis=1)

In [15]:
X = data.drop('Transported',axis=1)
y = data['Transported']
X_train,X_test,y_train,y_test = train_test_split(X,y,shuffle=True,test_size=0.0625)

In [16]:
from sklearn.preprocessing import StandardScaler

In [17]:
# ss = StandardScaler()
# ss.fit(X_train)
# X_train = ss.transform(X_train)
# X_test = ss.transform(X_test)

In [18]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

In [19]:
params = {
    'degree':[1,2,3,4,5],
    'gamma':['scale','auto'],
    'cache_size':[100,200,300,400,500,750,1000,1250,2500,5000,7500,10000],
    'verbose':[18],
    "max_iter":[100,200,300,400,500,750,1000,1250,2500,5000,7500,10000]
}

In [ ]:
model = SVC()
model.get_params().keys()
grid_search = GridSearchCV(estimator=model,param_grid=params,cv=5)
grid_search.fit(X_train,y_train)
grid_search.best_params_
grid_search.score(X_test, y_test)

[LibSVM]WARN: libsvm Solver reached max_iter
optimization finished, #iter = 100
obj = -199.948903, rho = -0.066737
nSV = 200, nBSV = 200
Total nSV = 200
[LibSVM]WARN: libsvm Solver reached max_iter
optimization finished, #iter = 100
obj = -199.993557, rho = -0.031103
nSV = 200, nBSV = 200
Total nSV = 200
[LibSVM]WARN: libsvm Solver reached max_iter
optimization finished, #iter = 100
obj = -199.909813, rho = -0.085287
nSV = 200, nBSV = 200
Total nSV = 200
[LibSVM]WARN: libsvm Solver reached max_iter
optimization finished, #iter = 100
obj = -199.994326, rho = -0.017590
nSV = 200, nBSV = 200
Total nSV = 200


/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


[LibSVM]WARN: libsvm Solver reached max_iter
optimization finished, #iter = 100
obj = -199.960390, rho = -0.075915
nSV = 200, nBSV = 200
Total nSV = 200
[LibSVM]WARN: libsvm Solver reached max_iter
optimization finished, #iter = 200
obj = -399.949868, rho = -0.075330
nSV = 400, nBSV = 400
Total nSV = 400
[LibSVM]

/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


WARN: libsvm Solver reached max_iter
optimization finished, #iter = 200
obj = -399.975663, rho = -0.067551
nSV = 400, nBSV = 400
Total nSV = 400
[LibSVM]WARN: libsvm Solver reached max_iter
optimization finished, #iter = 200
obj = -399.900610, rho = -0.096840
nSV = 400, nBSV = 400
Total nSV = 400
[LibSVM]WARN: libsvm Solver reached max_iter
optimization finished, #iter = 200
obj = -399.992576, rho = -0.021271
nSV = 400, nBSV = 400
Total nSV = 400


...

/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


WARN: libsvm Solver reached max_iter
optimization finished, #iter = 500
obj = -999.969626, rho = -0.095748
nSV = 1000, nBSV = 1000
Total nSV = 1000
[LibSVM]

/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


WARN: libsvm Solver reached max_iter
optimization finished, #iter = 500
obj = -999.943650, rho = -0.118653
nSV = 1000, nBSV = 1000
Total nSV = 1000
[LibSVM]

/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


In [ ]:
grid_search.best_params_

In [ ]:
# model = SVC(kernel='linear')
# model.fit(X_train,y_train)
# model.score(X_test, y_test)

In [ ]:
# preds = model.predict(np.array(X_test))

In [ ]:
# np.mean(cross_val_score(model,np.array(X_test),np.array(y_test),cv=25))

In [ ]:
# classification_report(preds,y_test)

In [ ]:
# np.mean(cross_val_score(model,np.array(X_train),np.array(y_train),cv=25))

In [ ]:
ids = test_data['PassengerId']
test_data['PassengerId'] = test_data['PassengerId'].astype(int)
test_data['Age'] = test_data['Age'].fillna(test_data['Age'].mean())
test_data['RoomService'] = test_data['RoomService'].fillna(test_data['RoomService'].mean())
test_data['FoodCourt'] = test_data['FoodCourt'].fillna(test_data['FoodCourt'].mean())
test_data['ShoppingMall'] = test_data['ShoppingMall'].fillna(test_data['ShoppingMall'].mean())
test_data['Spa'] = test_data['Spa'].fillna(test_data['Spa'].mean())
test_data['VRDeck'] = test_data['VRDeck'].fillna(test_data['VRDeck'].mean())
new_transported = []
for transport in test_data['CryoSleep']:
    if transport is True:
        new_transported.append(0)
    elif transport is False:
        new_transported.append(1)
    else:
        new_transported.append(None)
test_data['CryoSleep'] = new_transported
test_data = test_data.drop(['PassengerId','VIP','FoodCourt','ShoppingMall'],axis=1)
idx = 0
labels = {}
for label in test_data['HomePlanet']:
    if label not in list(labels.keys()):
        idx += 1
        labels[label] = idx
new_data = []
for i in test_data['HomePlanet']:
    new_data.append(labels[i])
test_data['HomePlanet'] = new_data
new_cabins = []
for c in test_data['Cabin']:
    if type(c) is float:
        new_cabins.append(None)
    else:
        c = c.split('/')[1]
        new_cabins.append(int(c))
test_data['Cabin'] = new_cabins
test_data['Cabin'] = test_data['Cabin'].fillna(test_data['Cabin'].mean())
test_data['CryoSleep'] = test_data['CryoSleep'].fillna(test_data['CryoSleep'].mean())
test_data = test_data.drop(['Destination','Name'],axis=1)

In [ ]:
test_data

In [ ]:
preds = model.predict(np.array(test_data))

In [ ]:
preds

In [ ]:
predictions = {
    "PassengerId":[],
    "Transported":[]
}

In [ ]:
labels = {0:True,1:False}

In [ ]:
for id,pred in zip(ids,preds):
    predictions['PassengerId'].append(id)
    predictions['Transported'].append(labels[pred])

In [ ]:
predictions = pd.DataFrame(predictions)

In [ ]:
predictions.to_csv('./save/svc-best.csv',index=False)